# Load data to InfluxDB

InfluxDB is a non relational data base specific for time series data sets. It has meany features, one is that can handle SQL like queries.

Here are shown the key steps to load the data una array of jsons to a measurement or **table** in InfluxDB

In [1]:
# Dependencies
import json
from influxdb import InfluxDBClient

Now we perform the actual connection to the db

In [2]:
client = InfluxDBClient(
    'influxdb',
    8086,
    'admin',
    'password',
    'db1')
client.create_database('starklink_db')
display(client.get_list_database())
client.switch_database('starklink_db')

[{'name': '_internal'}, {'name': 'db1'}, {'name': 'starklink_db'}]

Now we load the json file and setup the payload structure to be load into this Influxdb.

In [4]:
starklink_datafile = 'starlink_historical_data.json'

with open(starklink_datafile, 'r') as data_file:
    json_data = data_file.read()

starklink_data = json.loads(json_data)

payload = [
        {
        'measurement': 'sat_pos_' + data['id'],
        'time': data['spaceTrack']['CREATION_DATE'],
        'fields': {
            'latitude': data['latitude'],
            'longitude': data['longitude']}
        }
    for data in starklink_data ]


As we can see here, the payload consist on a list of dictionaries, where the key fields on these dictionaries are

    1. measurement: this is like the SQL DB where all the data will be allocated inside the DB.
    2. time: this is a ISO8601 or similar datetime string
    3. fields: points to a dict of the actual pair key-value of data stored
  
As an example


In [11]:
display(payload[100])

{'measurement': 'sat_pos_5eed7714096e590006985671',
 'time': '2021-01-26T14:26:10',
 'fields': {'latitude': -46.26538641856343, 'longitude': 13}}

Now with the payload constructed, we can do the actual write to the DB and then query for these measurements

In [6]:
client.write_points(payload)

True

In [12]:
client.query('select * from sat_pos_5eed7714096e590006985671')

ResultSet({'('sat_pos_5eed7714096e590006985671', None)': [{'time': '2021-01-21T06:26:10Z', 'latitude': 8.984105408081676, 'longitude': 136}, {'time': '2021-01-26T06:26:10Z', 'latitude': 8.984105408081676, 'longitude': 174}, {'time': '2021-01-26T14:26:10Z', 'latitude': -46.26538641856343, 'longitude': 13}]})